In [16]:
from src.data.data_loader import BrisT1DDataLoader

# Use 02, 03, 04
patient = "p02"

# Split up data into day and night
loader = BrisT1DDataLoader(use_cached=True)

train_df = loader.train_data[loader.train_data["p_num"] == patient]
test_df = loader.validation_data[loader.validation_data["p_num"] == patient]

TIME_STEP_SIZE = (
    train_df["datetime"].iloc[1] - train_df["datetime"].iloc[0]
).components.minutes


if TIME_STEP_SIZE != 5 and TIME_STEP_SIZE != 15:
    error = """
    First time step is not 5 or 15 minutes. Look at the most common time step size.
    """

## Calulcate CR and ISF
 - CR: carb/insulin ratio (how many carbs a insulin can cover)
     - Need to find out a large dose of insulin with no food announcmeent
 - ISF: blood sugar/insulin ratio (how much glucose drop for each unit of insulin)
     - Find out window where there is both insulin and food announcement 

Do we need to find out 5 windows for each patient? Like ISF for early morn, breakfast and so on....
Probably would be hard to figure that out?



```
{
    "offsetIndex": 0,
    "startOffsetTt": 14400,
    "name": "Early morn",
    "carbRatio": 13.5,
    "isf": 85,
    "insulinResistance": 1,
    "target": 100,
    "dateCreated": "2025-01-30T00:25:47.118865+00:00",
    "groupId": 10
},
{
    "offsetIndex": 1,
    "startOffsetTt": 23400,
    "name": "Breakfast",
    "carbRatio": 11.5,
    "isf": 85,
    "insulinResistance": 1,
    "target": 105,
    "dateCreated": "2025-01-30T00:25:47.118865+00:00",
    "groupId": 10
},
{
    "offsetIndex": 2,
    "startOffsetTt": 37800,
    "name": "Lunch",
    "carbRatio": 13.5,
    "isf": 18,
    "insulinResistance": 1,
    "target": 110,
    "dateCreated": "2025-01-30T00:25:47.118865+00:00",
    "groupId": 10
},
{
    "offsetIndex": 3,
    "startOffsetTt": 52200,
    "name": "Aft&Dinner",
    "carbRatio": 13.5,
    "isf": 85,
    "insulinResistance": 1,
    "target": 110,
    "dateCreated": "2025-01-30T00:25:47.118865+00:00",
    "groupId": 10
},
{
    "offsetIndex": 4,
    "startOffsetTt": 79200,
    "name": "Night",
    "carbRatio": 13.5,
    "isf": 85,
    "insulinResistance": 1,
    "target": 110,
    "dateCreated": "2025-01-30T00:25:47.118865+00:00",
    "groupId": 10
}
```